In [ ]:
#installing streamlit and pyngrok
!pip -q install streamlit
!pip -q install pyngrok

     |████████████████████████████████| 9.7 MB 5.2 MB/s 
     |████████████████████████████████| 111 kB 49.0 MB/s 
     |████████████████████████████████| 76 kB 4.5 MB/s 
     |████████████████████████████████| 4.3 MB 38.4 MB/s 
     |████████████████████████████████| 181 kB 56.0 MB/s 
     |████████████████████████████████| 164 kB 61.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 128 kB 57.0 MB/s 
     |████████████████████████████████| 793 kB 57.3 MB/s 
     |████████████████████████████████| 380 kB 52.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.9.1 which is incompatible.
google-colab 1.0.0 requires ipytho

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 66 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py

from pyngrok import ngrok 
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd
import numpy as np
import streamlit as st
import lightgbm as lgb
import time

st.title('To predict how much energy a building will consume')
st.header('Meter Reading Prediction')
selectedFile= st.file_uploader("Select a Query Point",type='csv')
if selectedFile is not None:
  st.subheader('Selected Query Point')
  query_point= pd.read_csv(selectedFile)
  st.write(query_point)
  my_bar = st.progress(0)
  # Reading pre-processed data from ftr files
  X_train_f= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_F.ftr')
  catboost_reg= joblib.load('/content/drive/MyDrive/PGDUOH/Project/catboost_reg_1200.sav')
  #handling missing values
  query_point['dew_temperature'].fillna(0, inplace=True)
  query_point['cloud_coverage'].fillna(0, inplace=True)
  query_point['precip_depth_1_hr'].fillna(0, inplace=True)
  # query_point['sea_level_pressure'].fillna(0, inplace=True)
  query_point['wind_direction'].fillna(0, inplace=True)
  query_point['wind_speed'].fillna(0, inplace=True)

  #Label encoding primary use and season column
  label_enc= LabelEncoder()
  label_enc.fit(X_train_f['primary_use'])
  query_point['primary_use']= label_enc.transform(query_point['primary_use'])

  label_enc.fit(X_train_f['season'])
  query_point['season']= label_enc.transform(query_point['season'])
  log_energy_usage= catboost_reg.predict(query_point)
  energy_usage= np.expm1(log_energy_usage)
  for percent_complete in range(100):
     time.sleep(0.1)
     my_bar.progress(percent_complete + 1)
  st.subheader('Predicted Meter Reading')
  st.write(energy_usage)

Writing app.py


In [ ]:
# !streamlit run app.py
# https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc
!ngrok authtoken 25aXq6on6c6HlBlD7pKjvIAzajI_3ia5qGwvo5daUBRQ7WCyC

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok as ngrok
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://0312-34-74-93-181.ngrok.io" -> "http://localhost:80"
2022-02-26 10:18:33.864 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
